ADP data by Steven Cunningham 06/16/17

This code is intened for take SonTek ADP data
- First determine seafloor by signal amplitude file ".a#". The signal should slowly decay through water, when decay is
  interupted with amplitude spike, then the signal is bouncing back from the floor. 
- Compare one bin to the previous until the next bin is larger than previous.
- Number of good profiles will be used on velocity data. 

In [516]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import csv


In [517]:
path = 'C:\\Users\\steve\\Desktop\\Thesis\\ADP files\\10.13.16'
sig_amp = 'SWC0610130945.a2'
VE_file = 'SWC0610130945.ve'
VN_file = 'SWC0610130945.vn'
df = pd.read_csv(sig_amp, delimiter='\s+', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,147,134,121,121,121,140,180,144,92,...,32,40,52,48,43,33,28,27,26,26
1,2,148,132,118,115,117,141,185,148,95,...,31,39,54,48,41,32,27,25,24,24
2,3,149,145,125,119,114,142,178,157,101,...,31,35,50,49,42,32,26,25,24,24
3,4,147,137,131,117,118,129,178,156,84,...,31,33,47,56,41,29,25,24,24,24
4,5,151,139,137,108,114,124,176,155,75,...,31,31,45,57,40,28,25,24,24,24


In [518]:
mean = np.mean(df, axis = 0)[1:] # average profiles across a depth bin
mean2 = pd.Series.to_frame(mean, 'average_profile') # transform to dataframe 
mean2["next_profile"] = mean2["average_profile"].shift(-1) # Shift down to commpare difference 
mean2["Nartural_decay"] = mean2["average_profile"] > mean2["next_profile"] # check if the next bin is greater than previous 
mean2["True_profile"] = mean2["Nartural_decay"].cumsum() # gives values to groups of true/false
mean2["Number"] = np.arange(23)[1:] # make a range of values 
mean2["good_profile"] = mean2["True_profile"] == mean2["Number"] # see if values assigned to true/false match number order  
print(mean2)

how_many=sum(mean2["good_profile"]) # how many good profiles are there. This will be used on velocity data 
mean3=mean2["average_profile"][:how_many+1] # get profile values for good bins. Add 1 because the .shift 


    average_profile  next_profile Nartural_decay  True_profile  Number  \
1        149.285714    139.642857           True             1       1   
2        139.642857    128.785714           True             2       2   
3        128.785714    119.428571           True             3       3   
4        119.428571    117.642857           True             4       4   
5        117.642857    135.571429          False             4       5   
6        135.571429    176.428571          False             4       6   
7        176.428571    149.428571           True             5       7   
8        149.428571     88.500000           True             6       8   
9         88.500000     57.071429           True             7       9   
10        57.071429     42.285714           True             8      10   
11        42.285714     34.857143           True             9      11   
12        34.857143     31.714286           True            10      12   
13        31.714286     35.857143     

Now calculate current speed by averaging velocities and direction 

In [519]:


VE = pd.read_csv(VE_file, delimiter='\s+', header=None)
VE.head() #depth = axis 0 = profile, axis 1 = depth
VN = pd.read_csv(VN_file, delimiter='\s+', header=None)
VN.head() #depth = axis 0 = profile, axis 1 = depth

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,4.4,3.0,3.3,4.3,4.1,0.4,3.9,6.0,4.9,...,1.5,8.5,4.5,13.3,15.7,-2.9,-7.4,-59.5,-85.5,-83.4
1,2,1.6,-1.5,0.7,0.6,1.2,-2.9,-2.0,1.7,-0.6,...,-4.6,-3.7,10.1,13.9,15.3,-4.2,-30.6,-65.5,-81.8,-79.2
2,3,1.6,-0.2,-0.4,4.6,5.0,3.5,-0.3,1.5,-2.0,...,0.0,-1.7,4.9,16.1,11.1,0.1,-14.3,-31.8,-40.1,-11.9
3,4,-0.7,1.8,2.1,-0.1,0.8,-0.8,1.0,2.9,1.9,...,-1.1,-1.1,5.9,16.5,13.0,1.2,-8.2,-25.5,-37.1,-39.0
4,5,0.5,-1.5,0.4,0.1,1.0,2.1,0.8,3.8,-1.0,...,2.2,3.5,9.1,11.2,12.3,10.5,12.5,30.2,33.9,26.5


In [520]:
VE=VE.iloc[:,1:how_many +1] # use how_many from aplitude to index good velocity profiles, add one because of 0 index
VE.head()
VN=VN.iloc[:,1:how_many +1] # use how_many from aplitude to index good velocity profiles, add one because of 0 index
VE.head()

,1,2,3,4
0,-6.1,-2.6,-1.8,-3.6
1,2.0,1.3,-0.1,1.4
2,0.9,0.0,-0.8,-0.1
3,-0.5,0.9,1.1,-1.9
4,-0.9,-0.1,-0.7,3.2


In [521]:
# average across same depth and round to .001
VEV= np.array(np.round(np.mean(VE, axis = 0), 3)) 
#VE["sign"]= VE['depth_bin_mean'] < 0
VNV = np.array(np.round(np.mean(VN, axis = 0), 3))
#VN["sign"]= VN['depth_bin_mean'] < 0
#VN["depth_bin_mean"] = VNX
NE_vector = (VEV**2 + VNV**2)**.5
print(NE_vector)
print(VNV, VEV)


[ 0.7131178   1.10708852  1.33541941  1.972926  ]
[-0.629 -0.014  1.064  1.929] [-0.336 -1.107 -0.807 -0.414]


In [522]:
North_sign = VNV > 0
East_sign = VEV >0
total_velocity = pd.DataFrame(NE_vector, columns = ["Velocity [cm/s]"])
print(total_velocity)

   Velocity [cm/s]
0         0.713118
1         1.107089
2         1.335419
3         1.972926


In [523]:
VNV = np.array(VNV) # change from DataFrame to np.array
VEV = np.array(VEV)
resultsN = []
for i in (VNV):    #make lists of the direction from positive or negative velocities 
    if i > 0:
        resultsN.extend("N")
    else:
        resultsN.extend("S")
print(resultsN)

['S', 'S', 'N', 'N']


In [524]:
resultsE = []
for i in (VEV):
    if i > 0:
        resultsE.extend("E")
    else:
        resultsE.extend("W")
print(resultsE)

['W', 'W', 'W', 'W']


In [525]:
#total_velocity['Depth'] = np.arange()
depth = np.arange(1,how_many+1)
depth = depth*.7
total_velocity['Depth (m)'] = depth
total_velocity['Direction1'] = resultsN
total_velocity['Direction2'] = resultsE
print(total_velocity)

   Velocity [cm/s]  Depth (m) Direction1 Direction2
0         0.713118        0.7          S          W
1         1.107089        1.4          S          W
2         1.335419        2.1          N          W
3         1.972926        2.8          N          W


In [526]:
total_velocity.to_csv(path +'/ADP_vector.csv')